In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split



In [2]:

images_path = '/content/drive/MyDrive/Phd/dataset/main'
annotations_path = '/content/drive/MyDrive/Phd/dataset/annotations'

input_height, input_width = 256, 256
num_classes = 2  # Background and brain

image_files = os.listdir(images_path)
annotation_files = os.listdir(annotations_path)

image_files.sort()
annotation_files.sort()

images = np.zeros((len(image_files), input_height, input_width, 3), dtype=np.float32)
annotations = np.zeros((len(annotation_files), input_height, input_width, 1), dtype=np.float32)

for i, (image_file, annotation_file) in enumerate(zip(image_files, annotation_files)):
    image_path = os.path.join(images_path, image_file)
    annotation_path = os.path.join(annotations_path, annotation_file)

    img = load_img(image_path, target_size=(input_height, input_width))
    images[i] = img_to_array(img) / 255.0

    annotation = load_img(annotation_path, target_size=(input_height, input_width), color_mode='grayscale')
    annotations[i] = img_to_array(annotation) / 255.0

X_train, X_temp, y_train, y_temp = train_test_split(images, annotations, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

# Define U-Net architecture
def unet(input_size=(input_height, input_width, 3)):
    inputs = Input(input_size)

    # Contracting path
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    # Bottom of the U
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Expanding path
    up5 = UpSampling2D((2, 2))(conv4)
    concat5 = concatenate([up5, conv3], axis=-1)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up6 = UpSampling2D((2, 2))(conv5)
    concat6 = concatenate([up6, conv2], axis=-1)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up7 = UpSampling2D((2, 2))(conv6)
    concat7 = concatenate([up7, conv1], axis=-1)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    # Output layer
    output = Conv2D(1, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=output)

    return model

model = unet()

In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

checkpoint = ModelCheckpoint('unet_segmentation_v2.keras', save_best_only=True)

history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val), callbacks=[checkpoint])

model.save('/content/drive/MyDrive/unet_segmentation_v2.keras')


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 256, 256, 64)         36928     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 128, 128, 64)         0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [1]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')

# Plot training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()





NameError: name 'model' is not defined